## Why is modeling helpful?

### 1.

Predictive modeling can show trends and patterns in the number of surgeries scheduled vs the actual number of surgeries performed.

### 2.

Predictive modeling can predict the number of actual surgeries performed on a given day, which can help with resource and staff allocation.

### 3.

With additional data, one can identify factors related to surgeon availability, seasonality, or health trends such as COVID-19 that affect the number of actual surgeries vs number of schedules surgeries; the hospital can thus prepare accordingly.

### 4.

Predictive modeling allows for automatic scheduling of surgeons, patients, nurses, and other important hospital staff around high volume or low volume periods to ensure maximizing resource utilization and minimizing waste.

### 5.

Predictive modeling can be used as a benchmark to see if the hospital's current policies are allowing it to function most efficiently as well as evaluate new changes.

In [1]:
import pandas as pd

dat = pd.read_csv('ElectiveSurgeryData.csv')

In [2]:
dat.head(5)

,SurgDate,DOW,T - 28,T - 21,T - 14,T - 13,T - 12,T - 11,T - 10,T - 9,T - 8,T - 7,T - 6,T - 5,T - 4,T - 3,T - 2,T - 1,Actual
0,10/10/2011,Mon,38,45,60,63,65,70,73,73,73,80,84,89,94,98,100,104,106
1,10/11/2011,Tue,35,47,65,68,78,82,82,82,86,89,92,95,99,99,99,114,121
2,10/12/2011,Wed,26,43,54,62,72,72,72,74,87,94,96,101,102,102,106,114,126
3,10/13/2011,Thu,28,48,65,70,72,72,72,82,87,91,94,94,94,97,98,103,114
4,10/14/2011,Fri,31,40,50,50,50,54,62,68,71,73,73,73,78,83,87,94,106
